In [12]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import os

### Question 1

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/06 20:42:54 WARN Utils: Your hostname, localhost resolves to a loopback address: 127.0.0.1; using 192.168.1.101 instead (on interface en0)
25/03/06 20:42:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 20:42:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.1'

### Question 2

In [5]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [6]:
main_dir = 'yellow_2024_04'
df = df.repartition(4)
df.write.mode("overwrite").parquet(main_dir)

In [9]:
parquet_files = [f for f in os.listdir(main_dir) if f.endswith('.parquet')]
for i in parquet_files:
    size_mb = os.path.getsize(f'{main_dir}/{i}') / (1024 * 1024)
    print(size_mb)

22.381834983825684
22.396079063415527
22.42396831512451
22.36391258239746


### Question 3

In [18]:
df.createOrReplaceTempView("trips")
spark.sql("""SELECT count(1) as cnt
             from trips
             WHERE DATE(tpep_pickup_datetime)  = '2024-10-15'
         """).show()

+------+
|   cnt|
+------+
|128893|
+------+



### Question 4

In [13]:
df_unix = df \
            .withColumn('pickup_unix', F.unix_timestamp(df.tpep_pickup_datetime) ) \
            .withColumn('dropoff_unix', F.unix_timestamp(df.tpep_dropoff_datetime) )
df_trip_dur = df_unix \
                    .withColumn('trip_hour', (df_unix.dropoff_unix - df_unix.pickup_unix) / 3600)
df_trip_dur.createOrReplaceTempView("trips_dur")
spark.sql("""
          SELECT VendorID, trip_hour
          FROM trips_dur
          ORDER BY trip_hour DESC
          LIMIT 1
          """).show()

+--------+------------------+
|VendorID|         trip_hour|
+--------+------------------+
|       2|162.61777777777777|
+--------+------------------+



### Question 6

In [15]:
df_lookup = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')
df_lookup_clean = df_lookup \
                        .withColumn("lo_id", F.col("LocationID").cast(IntegerType()) )
df_lookup_clean.createOrReplaceTempView("lookup")

In [16]:
spark.sql("""
         SELECT l.zone, COUNT(*)
         FROM trips as t
         LEFT JOIN lookup as l
         ON t.PULocationID = l.lo_id
         GROUP BY l.zone
         ORDER BY COUNT(*)
         LIMIT 1
         """).show(truncate = False)

+---------------------------------------------+--------+
|zone                                         |count(1)|
+---------------------------------------------+--------+
|Governor's Island/Ellis Island/Liberty Island|1       |
+---------------------------------------------+--------+

